# Generate figures for publication
_[work in progress]_

##  Tactile stimulation examples

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuroprocessing.imagingtrials import ImagingTrialLoader
from skimage.exposure import rescale_intensity
from skimage.filters import gaussian
from skimage.util import montage


def plot_STA_img_and_dff(imaging_trial,
                         trial_ROI,
                         show_montage = True,
                         flipLR = True,
                         bleach_subtract = True,
                         **dff_plot_kwargs):
    """
    Generate a montage or max projection image of the stimulus-triggered average (STA) of the 
    imaging trial, along with the DF/F trace of the trial.
    """
    mask = imaging_trial.load_mask()
    _, t, dff = imaging_trial.get_sta_stack(1, 5, .5, trial_ROI)
    sta_df, _, _ = imaging_trial.get_sta_stack(1, 5, .5)

    # remove linear trend from dff (remnants of bleaching)
    # only fit first and last 3 elements
    if bleach_subtract:
        dff = dff - np.polyval(np.polyfit(np.array([t[:3], t[-3:]]).flatten(),
                                        np.array([dff[:3], dff[-3:]]).flatten(), 1),
                                        t)

    # avreage over trials
    sta_df = sta_df.mean(axis=0)
    # rescale to (0, 1), ignoring masked out areas
    sta_df = rescale_intensity(sta_df,
                               in_range=(sta_df[sta_df>0].min(), sta_df.max()),
                               out_range=(0, 1),
                               )

    # add xy blur to sta_df image
    sta_df = gaussian(sta_df, sigma=2, channel_axis=0)

    # set everything outside the mask to 0
    sta_df = sta_df * mask

    if show_montage:
        f, axs = plt.subplots(nrows=1, ncols=2, figsize=(8,5))
        if flipLR:
            sta_df = np.flip(sta_df, axis=2)
        axs[0].imshow(montage(sta_df,
                        fill = 0,
                        padding_width = 20,
                        rescale_intensity=False,
                        grid_shape= None,
                        ),
                    cmap='inferno',
                    )

    else:
        # max projection over time
        sta_df_max = sta_df.max(axis=0)

        f, axs = plt.subplots(1, 2, figsize=(5, 3))
        axs[0].imshow(sta_df_max, cmap='inferno', aspect='auto')

        if flipLR:
            axs[0].invert_xaxis()

    axs[0].axis('off')
    axs[1].plot(t, dff)
    axs[1].set_xlabel('Time (s)')
    axs[1].set_ylabel('dF/F')
    axs[1].set(**dff_plot_kwargs)

    f.suptitle(imaging_trial)
    plt.tight_layout()


# Define ROIs for the LHL and RHL in the somatosensory cortex (SSC)
ssc_LHL_roi = {
    "center": (125,175),  # (x, y)
    "width": 80,
    "height": 80
}
ssc_RHL_roi = {
    "center": (265,175),  # (x, y)
    "width": 80,
    "height": 80
}

ssc_LHL_lido_roi = {
    "center": (175,225),  # (x, y)
    "width": 60,
    "height": 60
}

### RHL example trial

In [ ]:
imaging_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
imaging_trials.filter(exp_dir='2024-02-29',# "2024-03-19",
                      limb='RHL',
                      rec_time='5min',
                     )
trial = imaging_trials.trials[0]

plot_STA_img_and_dff(trial,
                     ssc_RHL_roi,
                     show_montage=False,
                     flipLR=True,
                     bleach_subtract=True,
                     ylim=[-0.03, 0.03])



### LHL example trial

In [ ]:
imaging_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
imaging_trials.filter(exp_dir='2024-02-29',# "2024-03-19",
                      limb='LHL',
                      rec_time='5min',
                      remainder='.*pt25pctISO_2'
                     )
trial = imaging_trials.trials[0]

plot_STA_img_and_dff(trial,
                     ssc_LHL_roi,
                     show_montage=False,
                     flipLR=True,
                     bleach_subtract=True,
                     ylim=[-0.03, 0.03])

### RFL example trial

In [ ]:
# RFL trial
ssc_RFL_roi = {
    "center": (335,210),  # (x, y)
    "width": 50,
    "height": 50
}
# RFL trial
imaging_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
imaging_trials.filter(exp_dir='2024-02-21',# "2024-03-19",
                      limb='RFL',
                      rec_time='5min',
                     )
trial = imaging_trials.trials[0]

plot_STA_img_and_dff(trial,
                     ssc_RFL_roi,
                     show_montage=False,
                     flipLR=True,
                     bleach_subtract=True,
                     ylim=[-0.03, 0.03])

# Lidocaine results
 * Tactile stimulation before and after localized lidocaine administration with saline as control

In [ ]:
imaging_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
imaging_trials.filter(exp_dir='2024-02-21',
                      limb='LHL',
                      rec_time='5min',
                     )


trial_pre = [t for t in imaging_trials if 'Zyla_5min_LHLstim_2son4soff_1pt25pctISO_deeper_1' in str(t)][0]
trial_post = [t for t in imaging_trials if 'Zyla_5min_LHLstim_2son4soff_1pt25pctISO_deeper_postlidocaine_2' in str(t)][0]
trial_sal = [t for t in imaging_trials if 'Zyla_5min_LHLstim_2son4soff_1pt25pctISO_deeper_postsaline_1' in str(t)][0]

plot_STA_img_and_dff(trial_pre, ssc_LHL_lido_roi, show_montage=True, bleach_subtract=True, ylim=[-0.03,.03])
plot_STA_img_and_dff(trial_post, ssc_LHL_lido_roi, show_montage=True, bleach_subtract=False, ylim=[-0.03,.03])
plot_STA_img_and_dff(trial_sal, ssc_LHL_lido_roi, show_montage=True, bleach_subtract=False, ylim=[-0.03,.03])


# Injection analysis
 * Stimulation always happens at t=60 seconds

## Capsaicin trials


### Different limbs trial (2024-03-19)

In [ ]:
def plot_montage_and_trace_pairs(imaging_trials, colors,
                                 montage_grid_shape = None, trace_ylim = None, trace_xlim = None):
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,5),
                            gridspec_kw={'height_ratios': [3, 1],
                                         'width_ratios': [1,1],
                                         'wspace' : .1,
                                         'hspace' : .05,
                                         }
                            )
    for trial,ax, line_color in zip(imaging_trials, axs.T, colors):
        # montage plot
        trial.plot_montage(s_start = 0,
                           s_end = 1200,
                           s_step = 30,
                           montage_grid_shape = montage_grid_shape,
                           ax=ax[0],
                           cmap='inferno',
                           aspect='equal')
        ax[0].axis('off')

        # trace plot
        t, trace = trial.load_trace()
        trace = trace - np.polyval(np.polyfit(t, trace, 1), t)
        trace = trace - trace[np.where(t >= 0)[0][0]]
        ax[1].plot(t, trace, color=line_color)
        ax[1].axvline(x=0, color='k', linestyle='--')
        ax[1].set_xlim(trace_xlim)
        ax[1].set_ylim(trace_ylim)
    axs[1,1].set_yticks([])

COLOR_SALINE = '#336699'
COLOR_HIST = '#33CC99'
COLOR_CAPSAICIN = '#FF3333'

# in this experiment, LHL was injected with vehicle, RHL with capsicin
inj_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
inj_trials.filter(exp_dir='2024-03-19',
                  limb='(L|R)HL$',
                 )
plot_montage_and_trace_pairs(inj_trials,
                             [COLOR_SALINE, COLOR_CAPSAICIN],
                             montage_grid_shape = None,
                             trace_ylim = [-1000,2500],
                             trace_xlim = [-50, 1400]
                             )


### Same limb trial (2024-03-20)

In [ ]:
# in this experiment, both injections were done in LHL
inj_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
inj_trials.filter(exp_dir='2024-03-18',# "2024-03-19",
                  limb='LHL',
                  injection_type='.*inj'
                 )

plot_montage_and_trace_pairs(inj_trials,
                             [COLOR_SALINE, COLOR_CAPSAICIN],
                             montage_grid_shape = None,
                             trace_ylim = [-1000,3000],
                             trace_xlim = [-50, 1400]
                             )


### Nape trial

In [ ]:
# in this experiment, both injections were done in LHL
inj_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
inj_trials.filter(exp_dir='2024-03-18',# "2024-03-19",
                  limb='nape',
                  injection_type='.*inj'
                 )

# reverse order to plot saline first
inj_trials = inj_trials[-1::-1]
plot_montage_and_trace_pairs(inj_trials,
                             [COLOR_SALINE, COLOR_CAPSAICIN],
                             montage_grid_shape = None,
                             trace_ylim = [-200,200],
                             trace_xlim = [-50, 1400]
                             )


## Histamine trials

### RHL trial (2024-02-29)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuroprocessing.imagingtrials import ImagingTrialLoader

# in this experiment, both injections were done in RHL
inj_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
inj_trials.filter(exp_dir='2024-02-29',# "2024-03-19",
                  limb='RHL$',
                  injection_type='.*inj'
                )

plot_montage_and_trace_pairs(inj_trials,
                             [COLOR_SALINE, COLOR_HIST],
                             montage_grid_shape = None,
                             trace_ylim = [-200,500],
                             trace_xlim = [-50, 1400]
                             )

### LHL trial (2024-02-29)

In [ ]:
# in this experiment, both injections were done in RHL
inj_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
inj_trials.filter(exp_dir='2024-02-29',# "2024-03-19",
                  limb='LHL$',
                  injection_type='.*inj'
                )

# reverse order to plot vehicle first
inj_trials = inj_trials[-1::-1]
plot_montage_and_trace_pairs(inj_trials,
                             [COLOR_SALINE, COLOR_HIST],
                             montage_grid_shape = None,
                             trace_ylim = [-200,500],
                             trace_xlim = [-50, 1400]
                             )

### RHL trial (2024-03-06)

In [ ]:
# in this experiment, both injections were done in RHL
inj_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
inj_trials.filter(exp_dir='2024-03-06',# "2024-03-19",
                  limb='RHL$',
                  injection_type='.*inj'
                 )

# reverse order to plot vehicle first
inj_trials = inj_trials[-1::-1]
plot_montage_and_trace_pairs(inj_trials,
                             [COLOR_SALINE, COLOR_HIST],
                             montage_grid_shape = None,
                             trace_ylim = [-300,500],
                             trace_xlim = [-50, 1400]
                             )

### LHL trial (2024-03-06)

In [ ]:
# in this experiment, both injections were done in RHL
inj_trials = ImagingTrialLoader('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections')
inj_trials.filter(exp_dir='2024-03-06',# "2024-03-19",
                  limb='LHL$',
                  injection_type='.*inj'
                )

plot_montage_and_trace_pairs(inj_trials,
                             [COLOR_SALINE, COLOR_HIST],
                             montage_grid_shape = None,
                             trace_ylim = [-300,500],
                             trace_xlim = [-50, 1400]
                             )